<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT23/blob/main/template-report-lab-X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Iterative methods**
**Nolwenn Deschand**

# **Abstract**


In this lab, we will deal with iterative methods for solving linear and nonlinear equations. Iterative methods are ways to deal with the problem of storage of nonzero components, which can make direct methods too expensive for large sparse systems. Iterative methods generate a sequence of approximated solutions, by reducing the error at each iteration. We will implement three different iterative methods: the Jacobi and Gauss-Seidel iterations for solving linear equations of the form $Ax = b$ and the Newton’s method for the nonlinear equation $f(x) = 0$.

---

Most of the algorithms are implemented from the pseudo-code present in the chapters 7 and 8 of the book *Methods in Computational Science*, from Johan Hoffman.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [1]:
"""This program is a lab report using the provided template"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# written by Nolwenn Deschand (ddeschand@kth.se)
# Template by Johan Hoffman


'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [9]:
# Load neccessary modules.
from google.colab import files

import numpy as np


# **Introduction**

 Iterative methods are useful to solve linear and nonlinear equations, for which direct methods can be too expensive. The main difference between direct and iterative methods, is that iterative methods do not have a fixed number of steps. Each step computes an approximation of the solution, which is improved in each step. The algorithm stops when the stopping criterion is satisfied, usually a tolerance that we set.


We will start by implement two iterative methods for solving the linear equation $Ax = b$ with A a matrix, x the solution and b a vector. We will use the Jacobi and Gauss-Seidel iterations, two methods than can under some conditions be equivalent to the left-preconditioned Richardson iteration. Instead of solving $Ax = b$, we will use the left precondition $BAx = Bx$, that will improve the rate of convergence. 


In a second part, we will implement Newton's method for solving nonlinear equations of the form $f(x) = 0$. Three iterative methods are available for this objective: the bisection, the fixed point iteration, and Newton’s method. The bisection methos is robust andd simple but its rate of convergence is slow. The fixed point iteration is a powerful general method that is straightforward to generalize to systems of nonlinear equations. Under some parameters, the fixed point iteration correspond to Newton’s method to enhance the rate of convergence. With Newton's method, we use the the derivative of the function in terms of the tangent line to estimate the new approximation.

# **Method**

**Implementation of the Jacobi iteration for $Ax = b$**

The Jacobi iteration is equivalent to Jacobi left preconditioned Richardson iteration with $α = 1$, that is solving the equation $BAx = Bb$ for a specific B. In the case of Jacobi iteration, we use for B a diagonal matrix composed of the inverse of the diagonal elements of A.

To implement this method, we will first implement the Richardson Iteration using the following algorithm from chapter 7: 
```
ALGORITHM 7.1. x = richardson_iteration(A, b, alpha). 
Input: n x n system matrix A, n vector b, parameter alpha. 
Output: solution vector x.
1: x=0
2: while norm(r)/norm(b) > TOL do
3:     r = matrix_vector_product(A, x) 
4:     r[:] = b[:] - r[:]
5:     x[:] = x[:] + alpha*r[:]
6: end while
7: return x
```
We will then implement the left preconditioned Richardson iteration and the Jacobi iteration based on the two previously implemented functions. 

The jacobi iteration will take as input a matrix A and vector b, and as output a vector x, the solution.

In [3]:
def richardson_iteration(A, b, alpha, tolerance):

  n = A.shape[0]
  x = np.zeros(n)
  r = b
  while np.linalg.norm(r)/np.linalg.norm(b) > tolerance:
    r = np.dot(A,x)
    r = b - r
    x = x + alpha * r
  return x

def left_preconditioned_richardson(A, B, b, alpha, tolerance):
  x = richardson_iteration(np.dot(B, A), np.dot(B,b), alpha, tolerance)
  return x

def jacobi_iteration(A, b, tolerance):
  Dinv = np.diag(1 / np.diag(A))
  x = left_preconditioned_richardson(A, Dinv, b, 1.0, tolerance)
  return x

**Implementation of the Gauss-Seidel iteration for $Ax = b$**

The Gauss-Seidel iteration is another method for solving linear equations. It is equivalent to a left preconditioned Ridchardson iteration with $B = L^{-1}$ and $α = 1$ (L is the lower triangular matrix obtained from the martix A by zeroing out all entries above the diagonal). Therefore, we can reuse the previously implemented functions ridchardson iteration and left preconditioned ridchardson. 

As for the Jacobi iteration, the Gauss-Seidel iteration will take as input a matrix A and vector b, and as output a vector x, the solution.


In [4]:
def gauss_seidel_iteration(A, b, tolerance):
  L = np.tril(A)
  Linv = np.linalg.inv(L)
  x = left_preconditioned_richardson(A, Linv, b, 1.0, tolerance)
  return x

**Implementation of Newton's method for scalar nonlinear equation $f(x) = 0$**

The implementation of Newton's method is based on the algorithm 8.2 in chapter 8 of the book:
```
ALGORITHM 8.2. x = newtons_method(f, x0). Input: a function f, an initial guess x0.
Output: approximate root x.
1: x = x0
2: while|f(x)|>TOL do
3: df = derivative(f, x)
4: x = x - f(x)/df
5: endwhile 
6: return x
```
The input of the function will be a scalar function f(x) and the output a real number number x, the solution. 


In [16]:
def derivative(f, x):
  h = 10**(-10)
  return (f(x + h) - f(x - h)) / (2 * h)

def newtons_method(f, x0, tolerance):
  x = x0
  while abs(f(x)) > tolerance:
    df = derivative(f,x)
    x = x - f(x)/df
  return x


# **Results**

**Implementation of the Jacobi iteration for $Ax = b$**

We will now test the Jacobi iteration. 

For the jacobi iteration, the convergence criterion is $∥I − D^{−1}A∥ < 1$, with D the matrix composed with the diagonal elements of A. It means that if the matrix x does not fulfill this convergence criterion, the jacobi iteration will not be able to converge and to produce a solution of the equation. In order to fulfill this convergence criterion, the matrix A has to be diagonally dominant. 

To test the Jacobi iteration, we will start by generating a random diagonally dominant matrix A. Then, we will test the convergence of the residual $|| Ax-b ||$, and $|| x-y ||$ for a manufactured solution y. The expected result is a value very close to zero in both cases.

In [14]:
n = np.random.randint(2,10)
A = np.random.rand(n,n);

while (np.linalg.norm(np.identity(n) - np.dot(np.diag(1 / np.diag(A)),A))) >= 1:
    A = A + np.diag(np.random.rand(n))


# TEST 1 : convergence of residual || Ax-b ||
b = np.random.rand(n)

x_jacobi1 = jacobi_iteration(A, b, 10**(-15))
residual1 = np.linalg.norm(np.dot(A,x_jacobi1)-b)
print("residual of ||𝐴𝑥−𝑏|| : ", residual1)


# TEST 2 : || x-y || for manufactured solution y 
x = np.random.rand(n)
b = np.dot(A, x)

x_jacobi2 = jacobi_iteration(A, b, 10**(-15))
residual2 = np.linalg.norm(np.dot(A,x_jacobi2)-b)

print("residual of ||𝑥−𝑦|| : ", residual2)


residual of ||𝐴𝑥−𝑏|| :  7.977012308035777e-16
residual of ||𝑥−𝑦|| :  1.0185048308013224e-14


The two residual values are very close to zero, as expected. The Jacobi iteration was able to produce a solution for $Ax = b$.

**Implementation of the Gauss-Seidel iteration for $Ax = b$**

As for Jacobi iteration, we will need to fulfill the convergence criterion for the Gauss-Seidel iteration to converge. 
This time, the convergence criterion is $∥I − L^{−1}A∥ < 1$, with L the lower triangular matrix obtained from the martix A by zeroing out all entries above the diagonal. 

The same generation of matrix A as the Jacobi iteration is enough to fulfill the convergence criterion.

Again, we will test the convergence of the residual $|| Ax-b ||$, and $|| x-y ||$ for a manufactured solution y. The expected result is also a value very close to zero in both cases.


In [15]:
n = np.random.randint(2,10)
A = np.random.rand(n,n);

while A is None or (np.linalg.norm(np.identity(n) - np.dot(np.linalg.inv(np.tril(A)),A))) >= 1:
    A = A + np.diag(np.random.rand(n))

# TEST 1 : convergence of residual || Ax-b ||
b = np.random.rand(n)

x_seidel1 = gauss_seidel_iteration(A, b, 10**(-15))
residual1 = np.linalg.norm(np.dot(A,x_seidel1)-b)
print("residual of ||𝐴𝑥−𝑏|| : ", residual1)


# TEST 2 : || x-y || for manufactured solution y 
x = np.random.rand(n)
b = np.dot(A, x)

x_seidel2 = gauss_seidel_iteration(A, b, 10**(-15))
residual2 = np.linalg.norm(np.dot(A,x_seidel2)-b)

print("residual of ||𝑥−𝑦|| : ", residual2)


residual of ||𝐴𝑥−𝑏|| :  2.7894008272968645e-16
residual of ||𝑥−𝑦|| :  6.661338147750939e-16


The residual values are very close to zero. We can consider that the implementation was successful for these test cases.

**Implementation of Newton's method for scalar nonlinear equation $f(x) = 0$**

To test the implementation of Newton's method, we will test convergence of residual $|f(x)|$ and $|x-y|$ for a manufactured solution y. We arnitrary choose a function f and a value x0 for the test.

For a correct implementation, we expect the residuals to be close to zero.


In [27]:
# TEST 1 : residual of |𝑓(𝑥)|
def f(x):
  return x**3+42

x = newtons_method(f, 2, 10**(-5))

print("residual of |𝑓(𝑥)| : ",abs(f(x)))

# TEST 2 : residual of |𝑥−𝑦|
def f2(x):
  return x**4 + x**3 + x**2 - 28
# expected solution :
y = 2
x = newtons_method(f2, 2, 10**(-5))
print("residual of |𝑥−𝑦| : ",abs(x-y))

residual of |𝑓(𝑥)| :  7.2869354994509195e-09
residual of |𝑥−𝑦| :  0


The results are close to zero as expected. The implementation seems to work for the simple test cases used here.

# **Discussion**

Finally, we have implemented three iterative methods, the Jacobi and Gauss-Seidel iteration to solve linear equations, and Newton's method to solve nonlinear equations. The implementations give the expected results for the simple test cases tried here. 

The efficiency of the methods is not tested here but it could be interesting to look at the rate of convergence and the number of iterations required for convergence for the different thresholds  (arbitrarily chosen at the moment), in order to adjust them according to the precision and computational time desired.